<a href="https://colab.research.google.com/github/jwoonge/ML-projects/blob/master/09/assignment_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
from torchvision import datasets, transforms

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,)),
])

data_path = './MNIST'
data_train = datasets.MNIST(root = data_path, train = True, download = True, transform = transform)
data_test = datasets.MNIST(root = data_path, train = False, download = True, transform = transform)
